# USV Processing notebook for running detection
Before running the cells in this notebook, please refer to this checklist:

- Activate usvprocessing environment prior to running code -- load from environment.yml
- Ensure the session files stored on your local device are:
    - in the correct format
    - located in the _wavfiles_ folder

cell outlining which wavfiles someone has in their wavfiles folder --> optional update at beginning of program
>

---

# Initial Setup
This section only needs to be run __once__
## Load imports

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import os

import usv_library as ul
import usv_class as uc
from usv_class import USV
import setup

Need cell explaining where things should be located

### Establish path names

In [ ]:
# Set path variables:
repo_path = os.getcwd()
USV_DATA_path = os.path.join(repo_path, 'USV_DATA')
wavfiles_path = os.path.join(repo_path, 'wavfiles')

## Set personal parameters

Run __once__ at the beginning of your validation/extraction session
> In this step you will:
- Set your rater ID (first and last inital)
- Set your desired frequency range

#### Select frequency range from the following options:
>A) 20 kHz (20,000 --> 30,000 Hz)  
>B) 40 kHz (40,000 --> 90,000 Hz)

In [ ]:
# Set rater ID
rater_id = setup.enter_rater_id()

# Set frequency range and parent directory
freq_range, freq_dir = setup.enter_freq_range()
USV_DATA_subpath = os.path.join(USV_DATA_path, freq_dir)

### View sessions with local data

In [1]:
# View session folders in wavfiles directory for easy access
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"session_folder":
              [subdir for subdir in os.listdir(wavfiles_path) if not subdir.endswith('.txt')]})

NameError: name 'pd' is not defined

#
---
<div style="text-align: center;">

> 
#### Only run the above code once! (Unless you are switching the frequency range you are validating or resetting your ID)

## Enter the session ID and verify WAV file exists

In [ ]:
# Session ID entry
session_id = setup.get_input("Session ID", "Enter the session name: ")
print(f"Session ID entered: {session_id} \n")

# Look for WAV file -- both detection and verification need it
wavfile = os.path.join(wavfiles_path, session_id, "Dodotronic Track.wav")
if not os.path.exists(wavfile):
    raise FileNotFoundError(f"WAV file not found, cannot continue. Missing: {wavfile}")
else:
    print(f"WAV file found: {wavfile}")
    print("Open in Audacity")

In [ ]:
# Check for a trial starts file (having this file is optional)
trial_starts, trial_starts_rater = setup.get_trial_starts(wavfiles_path, session_id)
if trial_starts is not None:
    print(f"File name: {trial_starts}")

## Run detection and/or labeling
#### First check for existing labels so detection is not rerun
- If labels already exist, move onto validation. If not, run detection.

In [ ]:
# Check if session has been processed before: check for labels / rater
labels_exist = setup.search_labels(USV_DATA_subpath, session_id)
existing_rater = setup.check_existing_rater(USV_DATA_subpath, session_id, rater_id)

In [ ]:
# Checks if labels exist before running USV detection
if not labels_exist:
    if trial_starts is not None:
        offset = setup.get_offset(trial_starts)
        print(f"Offset applied: {offset} seconds")
        usv = USV(wavfile, session_id, trial_starts, offset)
    else:
        usv = USV(wavfile, session_id)

    usv.run_setup(freq_range, USV_DATA_path, freq_dir) 
    usv.run_usv_detection(freq_range)

In [ ]:
if not existing_rater:
    raise Exception(f"{session_id} has already been rated by {rater_id}.")

# Runs sorting/validation on USV data (preexisting or newly detected)
session_data = os.path.join(USV_DATA_subpath, session_id)
usv_csv = os.path.join(session_data, f"{session_id}_USV.csv")
call_lst = ul.validate_usv(usv_csv, freq_dir)
pd.DataFrame(call_lst).to_csv(os.path.join(session_data, f"{session_id}_USV_rated_{rater_id}.csv"), index=False)
print("USV validation/sorting complete. Rated file added to session folder.")

# Finishing Protocol
At the end of a detection/validation session, please push new files to GitHub. 
> __NOTE:__ You should not `git push` your results every time you score, just `git push` at the end of your session.  

The pipeline is outlined here:
